In [11]:
import torch
import torch.nn as nn
import math
import time

device = "cuda"
torch.set_grad_enabled(False)

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

print("CUDA:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))

CUDA: True
GPU: NVIDIA RTX 4000 Ada Generation


In [12]:
class Config:
    vocab_size = 32000
    hidden_size = 768
    num_heads = 12
    num_layers = 6
    max_seq_len = 32768  # allow long contexts

config = Config()

In [13]:
## Paged KV cache

class PagedKVCache:
    def __init__(self, batch_size, num_heads, head_dim,
                 page_size=256, device="cuda"):
        self.batch_size = batch_size
        self.num_heads = num_heads
        self.head_dim = head_dim
        self.page_size = page_size
        self.device = device

        # List of allocated pages
        self.k_pages = []
        self.v_pages = []

        self.cur_pos = 0  # total tokens stored

    def _allocate_page(self):
        k_page = torch.zeros(
            self.batch_size,
            self.num_heads,
            self.page_size,
            self.head_dim,
            device=self.device
        )

        v_page = torch.zeros_like(k_page)

        self.k_pages.append(k_page)
        self.v_pages.append(v_page)

    def update(self, new_k, new_v):
        B, H, T, D = new_k.shape

        for t in range(T):
            page_idx = self.cur_pos // self.page_size
            offset = self.cur_pos % self.page_size

            if page_idx >= len(self.k_pages):
                self._allocate_page()

            self.k_pages[page_idx][:, :, offset, :] = new_k[:, :, t, :]
            self.v_pages[page_idx][:, :, offset, :] = new_v[:, :, t, :]

            self.cur_pos += 1

        # Return concatenated view
        k = torch.cat(self.k_pages, dim=2)[:, :, :self.cur_pos, :]
        v = torch.cat(self.v_pages, dim=2)[:, :, :self.cur_pos, :]

        return k, v

In [14]:
class PageStreamingAttention(nn.Module):
    def __init__(self, hidden_size, num_heads):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = hidden_size // num_heads

        self.q_proj = nn.Linear(hidden_size, hidden_size)
        self.k_proj = nn.Linear(hidden_size, hidden_size)
        self.v_proj = nn.Linear(hidden_size, hidden_size)
        self.out_proj = nn.Linear(hidden_size, hidden_size)

    def forward(self, x, kv_cache=None):
        B, T, C = x.shape

        q = self.q_proj(x)
        k = self.k_proj(x)
        v = self.v_proj(x)

        q = q.view(B, T, self.num_heads, self.head_dim).transpose(1,2)
        k = k.view(B, T, self.num_heads, self.head_dim).transpose(1,2)
        v = v.view(B, T, self.num_heads, self.head_dim).transpose(1,2)

        if kv_cache is not None:
            kv_cache.update(k, v)

        # Streaming attention over pages
        scale = 1.0 / math.sqrt(self.head_dim)

        output = torch.zeros_like(q)

        for page_idx in range(len(kv_cache.k_pages)):
            k_page = kv_cache.k_pages[page_idx]
            v_page = kv_cache.v_pages[page_idx]

            # Only consider valid tokens in last page
            if page_idx == len(kv_cache.k_pages) - 1:
                valid_tokens = kv_cache.cur_pos % kv_cache.page_size
                if valid_tokens == 0:
                    valid_tokens = kv_cache.page_size
                k_page = k_page[:, :, :valid_tokens, :]
                v_page = v_page[:, :, :valid_tokens, :]

            scores = torch.matmul(q, k_page.transpose(-2, -1)) * scale
            attn = torch.softmax(scores, dim=-1)
            output += torch.matmul(attn, v_page)

        output = output.transpose(1,2).contiguous().view(B, T, C)
        return self.out_proj(output)

In [15]:
class PagedKVCache:
    def __init__(self, batch_size, num_heads, head_dim,
                 page_size=256, device="cuda"):
        self.batch_size = batch_size
        self.num_heads = num_heads
        self.head_dim = head_dim
        self.page_size = page_size
        self.device = device

        self.k_pages = []
        self.v_pages = []
        self.cur_pos = 0

    def _allocate_page(self):
        k_page = torch.zeros(
            self.batch_size,
            self.num_heads,
            self.page_size,
            self.head_dim,
            device=self.device
        )
        v_page = torch.zeros_like(k_page)

        self.k_pages.append(k_page)
        self.v_pages.append(v_page)

    def update(self, new_k, new_v):
        B, H, T, D = new_k.shape

        for t in range(T):
            page_idx = self.cur_pos // self.page_size
            offset = self.cur_pos % self.page_size

            if page_idx >= len(self.k_pages):
                self._allocate_page()

            self.k_pages[page_idx][:, :, offset, :] = new_k[:, :, t, :]
            self.v_pages[page_idx][:, :, offset, :] = new_v[:, :, t, :]

            self.cur_pos += 1

        # Concatenate only once per forward
        k = torch.cat(self.k_pages, dim=2)[:, :, :self.cur_pos, :]
        v = torch.cat(self.v_pages, dim=2)[:, :, :self.cur_pos, :]

        return k, v

In [16]:
class TransformerBlock(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln1 = nn.LayerNorm(config.hidden_size)
        self.attn = PageStreamingAttention(config.hidden_size, config.num_heads)
        self.ln2 = nn.LayerNorm(config.hidden_size)
        self.mlp = nn.Sequential(
            nn.Linear(config.hidden_size, 4 * config.hidden_size),
            nn.GELU(),
            nn.Linear(4 * config.hidden_size, config.hidden_size)
        )

    def forward(self, x, kv_cache=None):
        x = x + self.attn(self.ln1(x), kv_cache)
        x = x + self.mlp(self.ln2(x))
        return x


class MiniTransformer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.embed = nn.Embedding(config.vocab_size, config.hidden_size)
        self.layers = nn.ModuleList([
            TransformerBlock(config)
            for _ in range(config.num_layers)
        ])
        self.ln_f = nn.LayerNorm(config.hidden_size)
        self.head = nn.Linear(config.hidden_size, config.vocab_size)

    def forward(self, input_ids, kv_caches=None):
        x = self.embed(input_ids)

        for i, layer in enumerate(self.layers):
            cache = kv_caches[i] if kv_caches else None
            x = layer(x, cache)

        x = self.ln_f(x)
        return self.head(x)

In [17]:
model = MiniTransformer(config).to(device)
model.eval()

print("Model ready")

Model ready


In [18]:
def print_gpu_memory():
    allocated = torch.cuda.memory_allocated() / 1e9
    reserved = torch.cuda.memory_reserved() / 1e9
    print(f"Allocated: {allocated:.2f} GB | Reserved: {reserved:.2f} GB")

In [19]:
def prefill_with_paging(seq_len):
    input_ids = torch.randint(0, config.vocab_size,
                              (1, seq_len),
                              device=device)

    kv_caches = [
        PagedKVCache(
            1,
            config.num_heads,
            config.hidden_size // config.num_heads,
            page_size=256,
            device=device
        )
        for _ in range(config.num_layers)
    ]

    torch.cuda.empty_cache()
    print_gpu_memory()

    _ = model(input_ids, kv_caches)

    print_gpu_memory()

In [20]:
prefill_with_paging(4096)
prefill_with_paging(8192)
prefill_with_paging(16384)

Allocated: 1.10 GB | Reserved: 1.18 GB
Allocated: 1.77 GB | Reserved: 2.20 GB
Allocated: 1.10 GB | Reserved: 1.18 GB
Allocated: 2.45 GB | Reserved: 3.21 GB
Allocated: 1.10 GB | Reserved: 1.18 GB
Allocated: 3.80 GB | Reserved: 5.24 GB


In [21]:
def prefill_with_paging(seq_len):
    input_ids = torch.randint(0, config.vocab_size,
                              (1, seq_len),
                              device=device)

    kv_caches = [
        PagedKVCache(
            1,
            config.num_heads,
            config.hidden_size // config.num_heads,
            page_size=256,
            device=device
        )
        for _ in range(config.num_layers)
    ]

    torch.cuda.empty_cache()
    print_gpu_memory()

    torch.cuda.synchronize()
    start = time.time()

    _ = model(input_ids, kv_caches)

    torch.cuda.synchronize()
    end = time.time()

    print(f"Prefill latency: {end - start:.4f}s")
    print_gpu_memory()

In [22]:
prefill_with_paging(4096)
prefill_with_paging(8192)
prefill_with_paging(16384)

Allocated: 1.10 GB | Reserved: 1.18 GB
Prefill latency: 1.3224s
Allocated: 1.77 GB | Reserved: 2.20 GB
Allocated: 1.10 GB | Reserved: 1.18 GB
Prefill latency: 2.8658s
Allocated: 2.45 GB | Reserved: 3.21 GB
Allocated: 1.10 GB | Reserved: 1.18 GB
Prefill latency: 6.7294s
Allocated: 3.80 GB | Reserved: 5.24 GB


In [23]:
def decode_with_paging(prefill_len, steps=256):
    input_ids = torch.randint(0, config.vocab_size,
                              (1, prefill_len),
                              device=device)

    kv_caches = [
        PagedKVCache(
            1,
            config.num_heads,
            config.hidden_size // config.num_heads,
            page_size=256,
            device=device
        )
        for _ in range(config.num_layers)
    ]

    # Prefill
    _ = model(input_ids, kv_caches)

    input_ids = torch.randint(0, config.vocab_size,
                              (1, 1),
                              device=device)

    torch.cuda.synchronize()
    start = time.time()

    for _ in range(steps):
        logits = model(input_ids, kv_caches)
        input_ids = torch.argmax(logits[:, -1, :],
                                 dim=-1,
                                 keepdim=True)

    torch.cuda.synchronize()
    end = time.time()

    print(f"Decode tok/s: {steps / (end - start):.2f}")

In [24]:
for ctx in [512, 1024, 2048, 4096]:
    decode_with_paging(ctx, 256)

Decode tok/s: 217.68
Decode tok/s: 181.51
Decode tok/s: 135.04
Decode tok/s: 89.36


In [31]:
class PageStreamingAttention(nn.Module):
    def __init__(self, hidden_size, num_heads):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = hidden_size // num_heads

        self.q_proj = nn.Linear(hidden_size, hidden_size)
        self.k_proj = nn.Linear(hidden_size, hidden_size)
        self.v_proj = nn.Linear(hidden_size, hidden_size)
        self.out_proj = nn.Linear(hidden_size, hidden_size)

    def forward(self, x, kv_cache=None):
        B, T, C = x.shape
    
        q = self.q_proj(x)
        k = self.k_proj(x)
        v = self.v_proj(x)
    
        q = q.view(B, T, self.num_heads, self.head_dim).transpose(1,2)
        k = k.view(B, T, self.num_heads, self.head_dim).transpose(1,2)
        v = v.view(B, T, self.num_heads, self.head_dim).transpose(1,2)
    
        scale = 1.0 / math.sqrt(self.head_dim)
    
        # 🔹 Case 1: No KV cache → standard full attention
        if kv_cache is None:
            attn = torch.matmul(q, k.transpose(-2, -1)) * scale
            attn = torch.softmax(attn, dim=-1)
            out = torch.matmul(attn, v)
            out = out.transpose(1,2).contiguous().view(B, T, C)
            return self.out_proj(out)
    
        # 🔹 Case 2: With KV cache → paged streaming
    
        kv_cache.update(k, v)
    
        output = torch.zeros_like(q)
    
        m_i = torch.full((B, self.num_heads, T),
                         -float("inf"), device=q.device)
        l_i = torch.zeros((B, self.num_heads, T),
                          device=q.device)
    
        for page_idx in range(len(kv_cache.k_pages)):
            k_page = kv_cache.k_pages[page_idx]
            v_page = kv_cache.v_pages[page_idx]
    
            if page_idx == len(kv_cache.k_pages) - 1:
                valid = kv_cache.cur_pos % kv_cache.page_size
                if valid != 0:
                    k_page = k_page[:, :, :valid, :]
                    v_page = v_page[:, :, :valid, :]
    
            scores = torch.matmul(q, k_page.transpose(-2, -1)) * scale
    
            block_max = scores.max(dim=-1).values
            new_m = torch.maximum(m_i, block_max)
    
            exp_scores = torch.exp(scores - new_m.unsqueeze(-1))
            exp_m_diff = torch.exp(m_i - new_m)
    
            l_i = exp_m_diff * l_i + exp_scores.sum(dim=-1)
            output = exp_m_diff.unsqueeze(-1) * output + torch.matmul(exp_scores, v_page)
    
            m_i = new_m
    
        output = output / l_i.unsqueeze(-1)
        output = output.transpose(1,2).contiguous().view(B, T, C)
    
        return self.out_proj(output)

In [40]:
def compare_attention(seq_len=128):
    input_ids = torch.randint(0, config.vocab_size,
                              (1, seq_len),
                              device=device)

    # Create one base model
    base_model = MiniTransformer(config).to(device)
    base_model.eval()

    # Clone it for streaming
    streaming_model = MiniTransformer(config).to(device)
    streaming_model.load_state_dict(base_model.state_dict())
    streaming_model.eval()

    # Replace attention in streaming model
    for i in range(config.num_layers):
        streaming_model.layers[i].attn = PageStreamingAttention(
            config.hidden_size,
            config.num_heads
        ).to(device)

    naive_out = base_model(input_ids)

    kv_caches = [
        PagedKVCache(
            1,
            config.num_heads,
            config.hidden_size // config.num_heads,
            page_size=256,
            device=device
        )
        for _ in range(config.num_layers)
    ]

    streaming_out = streaming_model(input_ids, kv_caches)

    diff = (naive_out - streaming_out).abs().max()
    print("Max difference:", diff.item())

In [41]:
compare_attention(128)

Max difference: 0.5442786812782288


In [42]:
prefill_with_paging(4096)
prefill_with_paging(8192)
prefill_with_paging(16384)

Allocated: 2.22 GB | Reserved: 2.34 GB
Prefill latency: 1.2965s
Allocated: 2.90 GB | Reserved: 3.31 GB
Allocated: 2.22 GB | Reserved: 2.34 GB
Prefill latency: 2.8709s
Allocated: 3.57 GB | Reserved: 4.28 GB
Allocated: 2.22 GB | Reserved: 2.34 GB
Prefill latency: 6.7553s
Allocated: 4.92 GB | Reserved: 6.33 GB


In [43]:
for ctx in [512, 1024, 2048, 4096]:
    decode_with_paging(ctx, 256)

Decode tok/s: 219.90
Decode tok/s: 181.48
Decode tok/s: 134.79
Decode tok/s: 88.98
